# 3 - Improving the way neural networks learn

The techniques we'll develop in this chapter inculde: a better choice of cost function, known as the cross-entropy cost function; four so-called "regularization" methods (L1 and L2 regularization, dropout, and artificial expansion of the training data), which make our networks better at generalizing beyond the training data; a better method for initializing the weights in the network; and a set of heuristics to help choose good hyper-parameters for the network.

Of course, we're only covering a few of the many, many techniques which have been developed for use in neural nets. **The philosophy is that the best entree to the plethora of available techniques is in-depth study of a few of the most important. Mastering those important techniques is not just useful in its own right, but will also deepen your understanding of what problems can arise when you use neural networks. That will leave you well prepared to quickly pick up other techniques, as you need them.**

## 3.1 - The cross-entropy cost function

As we discussed in last chapter (section 2.4), a weight will learn slowly if either the input neuron is low-activation($a^{l-1}_k\approx0$),or if the output neuron has saturated, i.e., is either high- or low-activation. This is the origin of the learning slowdown.

### 3.1.1 - Introducing the cross-entropy cost function

How can we address the learning slowdown? It turns out that we can solve the problem by replacing the quadratic cost with a different cost function, known as the cross-entropy. You may doubt about this. It's intuitively that we should replace the activation function $\sigma$ with another activation function instead of replacing the quadratic cost. You will undertand this soon.

We define the cross-entropy cost function by

$$C = -\frac{1}{n}\sum_{x}[y\;lna + (1-y)ln(1-a)], \tag{3.1}$$

where n is the total number of items of training data, the sum is over all training inputs, $x$, and $y$ is the corresponding desired output.

It's not obvious that the expression (3.1) fixes the learning slowdown problem. In fact, frankly, it's not even obvious that it makes sense to call this a cost function! However, it's a good news that the Equation (3.1) satisfies the two assumptions of backpropagation algorithm in last chapter (section 2.2). Before addressing the learning slowdown, let's see in what sense the cross-entropy can be interpreted as a cost function.

Two properties in particular make it reasonable to interpret the cross-entropy as a cost function.

* Non-negative
* If the neuron's actual output is close to the desired output, then the cross-entropy will be close to zero.

**These are both properties we'd intuitively expect for a cost function.** So that's good news for the cross-entropy, and we understand why call cross-entropy a cost function. Then we address the problem of learning slowdown. To see this, let's compute the partial derivative of the cross-entropy cost with respect to the weights. We substitute $a = \sigma(z)$ into (3.1), and apply the chain rule twice, obtaining:

$$\frac{\partial C}{\partial w_j} = -\frac{1}{n}\sum_{x}(\frac{y}{\sigma(z)} - \frac{1-y}{1-\sigma(z)})\sigma'(z)x_j =\frac{1}{n}\sum_{x}\frac{\sigma'(z)x_j}{\sigma(z)(1-\sigma(z))}(\sigma(z) - y). \tag{3.2}$$

Note that $\sigma'(z) = \sigma(z)(1-\sigma(z))$, and it simplifies to become:

$$\frac{\partial C}{\partial w_j} = \frac{1}{n}\sum_{x}x_j(\sigma(z) - y). \tag{3.3}$$

This is a beautiful expression. It tells us that the rate at which the weight learns is controlled by $\sigma(z)-y$. The larger the error, the faster the neuron will learn. This is just what we'd intuitively expect. **In particular, it avoids the learning slowdown caused by the $\sigma'(z)$ term. When we use the cross-entropy, the $\sigma'(z)$ term gets canceled out, and we no longer need worry about it being small.**

In a similar way, we can compute the partial derivative for the bias. I won't go through all the details again, but you can easily verify that

$$\frac{\partial C}{\partial b} = \frac{1}{n}\sum_{x}(\sigma(z) - y). \tag{3.4}$$

Again, this avoids the learning slowdown caused by the $\sigma'(z)$ term.

### 3.1.2 - Using the cross-entropy to classify MNIST digits

**As discussed above, it's not possible to say precisely what it means to use the "same" learning rate when the cost function is changed.** So learning rate should be changed as the cost function is changed.

Let's look at the case where we use 100 hiden neurons, the cross-entropy, and otherwise keep the parameters the same. In this case we obtain an accuracy of 96.82 percent. That's a substantial improvement over the results from Chapter 1, where we obtained a classification accuracy of 96.59 percent, using the quadratic cost.

**Why go to so much effort when it gives only a small improvement to our MNIST results?**

Part of the reason is that the cross-entropy is a widely-used cost function, and so is worth understanding well. But the more important reason is that neuron saturation is an important problem in neural nets. And so I've discussed the cross-entropy at length because it's a good laboratory to begin understanding neuron saturation and how it may be addressed.

### 3.1.3 What does the cross-entropy mean? Where does it come from?

Our discussion of the cross-entropy has focused on algebraic analysis and practical implementation. That's useful, but it leaves unanswered broader conceptual questions.

**Let's begin with the question that what could have motivated us to think up the cross-entropy in the first place?**

Suppose we'd discovered the learning slowdown described earlier, and understood that the origin was the $\sigma'(z)$ term. So we wonder if it's possible to choose a cost function so that the $\sigma'(z)$ term disappeared. In that case, the cost $C = C_x$ for a single training example $x$ would satisfy

$$\frac{\partial C}{\partial w_j} = x_j(a-y)\tag{3.17}$$

$$\frac{\partial C}{\partial b} = (a-y). \tag{3.18}$$

Note that from the chain rule we have

$$\frac{\partial C}{\partial b} = \frac{\partial C}{\partial a}\sigma'(z). \tag{3.19}$$

Using $\sigma'(z) = \sigma(z)(1-\sigma(z)) = a(1-a)$ the last euqation becomes

$$\frac{\partial C}{\partial b} = \frac{\partial C}{\partial a}a(1-a). \tag{3.20}$$

Comparing to Equation 3.18 we obtain

$$\frac{\partial C}{\partial a} = \frac{a-y}{a(1-a)}. \tag{3.21}$$

Integrating this expression with respect to a gives ($C = \int{\frac{a-y}{a(1-a)}}\,{\rm d}a$)

$$C = -[ylna + (1-y)ln(1-a)] + constant, \tag{3.22}$$


for some constant of integration. This is the contribution to the cost from a single training example, $x$. To get the full cost function we must average over training examples, obtaining

$$C = -\frac{1}{n}\sum_{x}[ylna + (1-y)ln(1-a)] + constant, \tag{3.23}$$

where the constant here is the average of the individual constants for each training example.

**The cross-entropy isn't something that was miraculously pulled out of thin air. Rather, it's something that we could have discovered in a simple and natural way.**

**What about the intuitive meaning of the cross-entropy? How should we think about it?**

I can't answer these questions in depth at present.

### 3.1.4 Softmax

Another approach to the problem of learning slowdown is based on what so-called $softmax$ layers of neurons. **The idea of softmax is to define a new type of output layer for our neural networks.** It begins in the same way as with a sigmoid layer, by forming the weighted input $z^L_j = \sum_k w^L_{jk}a^{L-1}_k + b^L_j$. **However, we don't apply the sigmoid function to get the output. Instead, in a softmax layer we apply the so-called softmax function to the $z^L_j$.** According to this function, the activation $a^L_j$ of the $j$-th output neuron is

$$a^L_j = \frac{e^{Z^L_j}}{\sum_k e^{Z^L_k}}. \tag{3.24}$$

To better understand Equation (3.24), suppose we have a network with four output neurons, and four corresponding weighted inputs, which we'll denote $z^L_1, z^L_2, z^L_3, $ and $z^L_4$. As we increase $z^L_4$, we'll see an increase in the corresponding output activation, $a^L_4$, and a decrease in the other output activation. Similarly, if we decrease $z^L_4$ then $a^L_4$ will decrease, and all the other output activations will increase.

**In fact, if we look closely, we'll see that in both cases the total change in the other activations exactly compensates for the change in $a^L_4$.** The reason is that the output activations are guaranteed to always sum up to 1, as we can prove using Equation (3.24) and a little algebra:

$$\sum_{j}a^L_j = \frac{\sum_j e^{Z^L_j}}{\sum_k e^{z^L_k}} = 1. \tag{3.25}$$

Equation (3.24) also implies that the output activations are all positive, since the exponential function is positive. Combining this with what we discussed above, we see that the output from the softmax layer is a set of positive numbers which sum up to 1. **In other words, the output from the softmax layer can be thought of as a probability distribution.**

By contrast, if the output layer was a sigmoid layer, then we certainly couldn't assume that the activations formed a probability distribution. And so with a sigmoid output layer we don't have such a simple interpretation of the output activations.